In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
#from matplotlib import subplo
import scipy.io as sio
from numpy import linalg as la
from scipy.spatial import Delaunay, distance
from scipy.stats import norm as normal
from PIL import Image, ImageDraw
import itertools
import argparse
import random
import copy
import json
import h5py
import multiprocessing
import datetime
from time import gmtime, strftime

In [2]:
def bilateral_filter( img, diameter, scolor, sspace, minthresh, ratio ):
    """
    Blurs the borders, in order to remove lines drawn in the image, or
    artifacts because the fabrication method.
    """
    bilateral = cv2.bilateralFilter(img, diameter, scolor, sspace)
    return bilateral


In [3]:
#img=cv2.imread('/scratch/data/PMD1464&1463-F31-2014.02.01-04.12.52_PMD1463_1_0091_lossless.jp2',-1)
#img8=(img/16).astype('uint8')

In [4]:
#img8=1*img8+10
#plt.figure(figsize=(20,20))
#plt.imshow(img8[5000:10000,17000:20000,])
#plt.show()

In [5]:
DataL=[]
with open('/home/xli/Documents/scratch/Marmoset_Process_Detection/M919filenames.txt') as f:
    #imglist = '/home/xli/Documents/scratch/Marmoset820MAT/' + f.readlines() + '.mat'
    for line in f:
        DataL.append('/home/xli/Documents/scratch/Marmoset919MAT/' + line.rstrip('\n') + '.mat')

In [6]:
def process_detect(file):
    print 'processing '+file
    img=h5py.File(file)
    img=np.asarray(img['img'])
    img=img.T
    img=cv2.convertScaleAbs(img)#convert to 8 bits for bilateral filter
    
    img_bf=bilateral_filter(img,10,50,50,20,6)
    
    #for i in range(36000):
        #for j in range(48000):
            #if np.max(img_bf[i,j,:]) != img_bf[i,j,1]:
                #img_bf[i,j,:] = 0
    A = np.max(img_bf, axis = 2)
    result = A != img_bf[:,:,1]
    img_bf[result] = 0
    #img_bf_g_pass=np.copy(img_bf)
    cstr1= img_bf[:,:,1] < img_bf[:,:,0]+20
    cstr2= img_bf[:,:,1] < img_bf[:,:,2]+20

    #print cstrl1.shape

    img_bf[cstr1] = 0
    img_bf[cstr2] = 0

    img_bf[img_bf<70]=0

    cv2.imwrite(file.split('.')[0]+'_process.jp2',img_bf)
    
    print file+' done'

In [7]:
len(DataL)

337

In [ ]:
p=multiprocessing.Pool(4)
p.map(process_detect,DataL)

processing /home/xli/Documents/scratch/Marmoset919MAT/M919-F100--_3_0300.jp2.mat
processing /home/xli/Documents/scratch/Marmoset919MAT/M919-F121--_3_0363.jp2.mat
processing /home/xli/Documents/scratch/Marmoset919MAT/M919-F13--_2_0038.jp2.mat
processing /home/xli/Documents/scratch/Marmoset919MAT/M919-F142--_2_0425.jp2.mat
/home/xli/Documents/scratch/Marmoset919MAT/M919-F13--_2_0038.jp2.mat done
processing /home/xli/Documents/scratch/Marmoset919MAT/M919-F132--_2_0395.jp2.mat
/home/xli/Documents/scratch/Marmoset919MAT/M919-F142--_2_0425.jp2.mat done
processing /home/xli/Documents/scratch/Marmoset919MAT/M919-F142--_3_0426.jp2.mat
/home/xli/Documents/scratch/Marmoset919MAT/M919-F121--_3_0363.jp2.mat done
processing /home/xli/Documents/scratch/Marmoset919MAT/M919-F12--_2_0035.jp2.mat
/home/xli/Documents/scratch/Marmoset919MAT/M919-F100--_3_0300.jp2.mat done
processing /home/xli/Documents/scratch/Marmoset919MAT/M919-F101--_3_0303.jp2.mat
/home/xli/Documents/scratch/Marmoset919MAT/M919-F132--_

In [7]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())

file=DataL[243]
img=h5py.File(file)
img=np.asarray(img['img'])
img=img.T
print strftime("%Y-%m-%d %H:%M:%S", gmtime())


2017-04-26 05:09:47
2017-04-26 05:10:03


In [8]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())
img_bf=bilateral_filter(img,10,50,50,20,6)
print strftime("%Y-%m-%d %H:%M:%S", gmtime())


2017-04-26 05:10:03
2017-04-26 05:10:34


In [9]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())

A = np.max(img_bf, axis = 2)
result = A != img_bf[:,:,1]
img_bf[result] = 0
print strftime("%Y-%m-%d %H:%M:%S", gmtime())


2017-04-26 05:10:34
2017-04-26 05:11:02


In [11]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())
img_bf[img_bf<60]=0
print strftime("%Y-%m-%d %H:%M:%S", gmtime())


2017-04-26 05:11:18
2017-04-26 05:11:26


In [13]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())
cv2.imwrite('/home/xli/Documents/scratch/Marmoset820P/test.jp2',img_bf)
print strftime("%Y-%m-%d %H:%M:%S", gmtime())

2017-04-26 05:14:25
2017-04-26 05:17:19


In [17]:
file.split('.')[0]+'_process.jp2'

'/home/xli/Documents/scratch/Marmoset820MAT/M820-F196--_3_0588_process.jp2'

In [ ]:
p=multiprocessing.Pool(3)
p.map(process_detect,DataL)

In [ ]:
p=multiprocessing.Pool()
for f in DataL[210:220]:
    p.apply_async(process_detect,args=f)
#p.close()
#p.join()

In [ ]:
p=multiprocessing.Pool(processes=80)

In [ ]:
p.apply_async(process_detect,tuple(DataL[210:220]))

In [ ]:
#multi test
def f(x):
    return x*x
p=multiprocessing.Pool(5)
k=p.apply_async(f, (2,))
k=p.apply_async(f, (4,))

k.get(timeout=1)

In [ ]:
img=h5py.File('/scratch/Marmoset_Process_Detection/M820-F196--_3_0588.mat')
img=np.asarray(img['img'])
img=img.T

In [ ]:
img_bf=bilateral_filter(img,10,50,50,20,6)

In [ ]:

img=h5py.File(DataL[200])
#img=np.asarray(img['img'])
img.keys()

In [ ]:
#cv2.imwrite('F196-BF-200.jp2',img_bf)

In [ ]:
img_bf.shape

In [ ]:
#only preserve G is biggest?
for i in range(36000):
    for j in range(48000):
        if np.max(img_bf[i,j,:]) != img_bf[i,j,1]:
            img_bf[i,j,:] = 0

In [ ]:
img_bf[1000,1000,]

In [ ]:
cv2.imwrite('F196-BF-G.jp2',img_bf)

In [ ]:
img_bf_g_pass=np.copy(img_bf)

img_bf_g_pass[img_bf_g_pass<70]=0



cv2.imwrite('F196-BF-G-pass-70.jp2',img_bf_g_pass)

In [ ]:
img_bf_g_pass[1000,1000,]

In [ ]:
img_bf_g_pass==img_bf